# Match entre empresas y  freelances mediante el índice de Jaccard  
## Depurado y adaptación de los datasets 

In [1]:
#librerías necesarias
import pandas as pd
import os
import sys
import numpy as np

Leer los datasets a depurar 

In [2]:
# El path donde estén tus csv
#path="C://Users//lopez//Documents//cursos//master BIG DATA//TFM"
path="C://Users//Usuario//Desktop//TFM//"
#os.getcwd()
os.chdir(path)
# creamos los dataframes de los csv de compañías y freelances.
# companies = pd.read_excel("companies.xlsx")
# freelances = pd.read_excel("freelances.xlsx")
companies = pd.read_csv("companies.csv")
freelances = pd.read_csv("freelances.csv")

Separación del dataset para únicamente conservar las filas importantes para la depuración 

In [77]:

companies_cols=["Company_Name","Industry of company","Prescriptive analytics business","Speech analytics business","Predictive Analytics business",
                "Big Data Business","Machine Learning based business","Highest education","Specialization of highest education",
                "Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Engineering",
                "Percent_skill_Leadership", "Percent_skill_Business Strategy"]

companies=companies[companies_cols]

freelances_cols=["Respondent","DeveloperType","FormalEducation","MajorUndergrad","LearningNewTech","ChangeWorld","RightWrongWay","CompetePeers",
                "DiversityImportant","ChallengeMyself","UnderstandComputers","InvestTimeTools","BuildingThings"]

freelances=freelances[freelances_cols]


## Funciones para crear los datasets finales que usaremos para aplicar el íncide de Jaccard

Función generica para convertir categorías a dummies

In [4]:
def col_to_dummies(df,col):
    """
    Remueve los NaN, no crea una categoría para ellos
    """
    df_out = pd.get_dummies(df,columns=[col], prefix=col)
    cols=df_out.columns.tolist()[1:]
    df_out[cols]=df_out[cols].astype(float)
    return df_out

 ## Funciones relacionadas con habilidades técnicas:

Match entre las columnas del dataset de compañías con las la columna Developer type del dataset de freelancers:                                                          
                                                     *Industry of Company
                                                     * Predictive Analytics Business
                                                     *Speech analytics Business
                                                     *Prescriptive analytics Business
                                                     *Machine leaning based Business
                                                  

In [72]:

def Tech_Skills_Freelancers(freelances,col_Respondent,developer_type):
    """
    Recategorizarión que sale de la col DeveloperType de freelances:
    Software Engineer, Machine Learning, Analytics, Data scientist
    Database or system administrator, Graphics Designer,
    Mobile developer, Web developer, Other
    
    """
    df_dev = freelances[[col_Respondent,developer_type]].copy()
    df_dev[developer_type]=df_dev[developer_type]
    df_dev[developer_type]=df_dev[developer_type].str.replace(';.*','')

    #df_dev.loc[(df_dev[developer_type] == "DevOps specialist")  ,developer_type] = "DevOps"
   
    df_dev.loc[(df_dev[developer_type] == "Graphics programming") | (df_dev[developer_type] == "Graphic designer") ,developer_type] = "Graphics Designer"
    df_dev.loc[(df_dev[developer_type] == "Embedded applications/devices developer") | (df_dev[developer_type] == "Desktop applications developer") |
               (df_dev[developer_type] == "Quality assurance engineer") |(df_dev[developer_type] == "DevOps specialist") ,developer_type] = "Software Engineer"
    df_dev.loc[(df_dev[developer_type] == "Database administrator") | (df_dev[developer_type] == "Systems administrator") ,developer_type] = "Database or system administrator"
    df_dev.loc[(df_dev[developer_type] == "Developer with a statistics or mathematics background")  ,developer_type] = "Analytics"
    df_dev.loc[(df_dev[developer_type] == "Machine learning Specialist"),developer_type] = "Machine Learning"
    df_out=col_to_dummies(df_dev,developer_type)
    return df_out

def recode_tech_skills_comp(df_companies,col_Company_Name,col_companies):
    """
    Recategorizarión que sale de la col Industry of Company de companies:
    Software Engineer, Analytics, Graphics Designer, Web developer, Mobile developer, other
    Como no existe la categoría Database or system administrator, nos creamos una columna a 0.
    
    """
    df_aux=companies[[col_Company_Name,col_companies]].copy()
    #df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(big data).*','Data scientist')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(analytics|cloud).*','Analytics')
    #df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(machine learning).*','Machine Learning')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(mobile).*','Mobile developer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(network|marketing|publishing|gaming|advertising).*','Graphics Designer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(software|cloud).*','software engineer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(E-commerce).*','Web developer')                                              
    #df_aux[col_companies]=df_aux[col_companies].fillna("sin info")
    df_aux.loc[(df_aux[col_companies] != "Data scientist") & (df_aux[col_companies] != "Analytics") &
               (df_aux[col_companies] != "Machine Learning") & (df_aux[col_companies] != "Mobile developer") &
               (df_aux[col_companies] != "Graphics Designer") & (df_aux[col_companies] != "software engineer") &
               (df_aux[col_companies] != "nan") & (df_aux[col_companies] != "Web developer"),col_companies] = "other"
    df_out=col_to_dummies(df_aux,col_companies)
    df_out["Database or system administrator"]=0.0
    return df_out


cols_companies_DS=["Company_Name","Prescriptive analytics business","Big Data Business","Speech analytics business","Predictive Analytics business"]
cols_companies_ML=["Company_Name","Machine Learning based business"]

def recode_tech_DS_ML_comp(df_companies,cols_companies,out_col_name):
    """
    Recodificamos la columnas asociadas a la categoria de ML o DS como una sóla con valores 0, 1 o nan.
    ds=Prescriptive analytics business,Big Data Business,Speech analytics business,Predictive Analytics business
    ml=Machine Learning based business

    """
    df_aux=companies[cols_companies].copy()
    for i in cols_companies[1:]:
        df_aux[i]=df_aux[i].replace('Yes',1)
        df_aux[i]=df_aux[i].replace('No',0)
        df_aux[i]=df_aux[i].replace('No Info',np.nan)

    df_aux[out_col_name] = df_aux[cols_companies[1:]].max(axis=1, skipna = True)
    df_out=df_aux[[cols_companies[0],out_col_name]]
    return df_out



Muestra del dataset de companias una vez aplicadas las funciones anteriores 

In [6]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_out=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_out,df_companies3,on='Company_Name',how='left')
df_companies_out1.head()

,Company_Name,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning
0,Company1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [88]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out1= df_freelance_out1[['Respondent','DeveloperType_Analytics','DeveloperType_Graphics Designer','DeveloperType_Mobile developer','DeveloperType_Web developer','DeveloperType_Other','DeveloperType_Software Engineer','DeveloperType_Database or system administrator','DeveloperType_Data scientist','DeveloperType_Machine learning specialist']]
df_freelance_out1.head()

,Respondent,DeveloperType_Analytics,DeveloperType_Graphics Designer,DeveloperType_Mobile developer,DeveloperType_Web developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Database or system administrator,DeveloperType_Data scientist,DeveloperType_Machine learning specialist
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


 ## Funciones relacionadas con educación:  
 Match entre la columna de companies Highest education con la de FormalEducation.
 Match entre la columna de companies Specialization of highest education con la de Major Undergraded.                                  

In [8]:

def recode_Highest_education(df_companies,col_Company_Name,col_companies):
    """
    Recodificación requisitos de nivel educativo que requieren las empresas
    """
    df_aux = df_companies[[col_Company_Name,col_companies]].copy()
    df_aux=col_to_dummies(companies[[col_Company_Name,col_companies]],col_companies)
    df_aux["under bachelor's degree"]=0.0
    return df_aux

def recode_FormalEducation(df_freelance,col_Respondent,col_freelance):
    """
    Recodificación nivel educativo de los desarrolladores
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[df_aux[col_freelance] == "Master's degree",col_freelance] = "Masters"
    df_aux.loc[(df_aux[col_freelance] == "Professional degree") |
               (df_aux[col_freelance] == "Bachelor's degree"),col_freelance] = "Bachelors"
    df_aux.loc[df_aux[col_freelance] == "Doctoral degree",col_freelance] = "PhD"
    df_aux.loc[df_aux[col_freelance] == "I prefer not to answer",col_freelance] = "No Info"
    df_aux.loc[(df_aux[col_freelance] == "Secondary school") |
              (df_aux[col_freelance] == "Some college/university study without earning a bachelor's degree") |
              (df_aux[col_freelance] == "Primary/elementary school") |
              (df_aux[col_freelance] == "I never completed any formal education")
               ,col_freelance] = "under bachelor's degree"
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out


def recode_MajorUndergrad(df_freelance,col_Respondent,col_freelance):
    """
    Recodificación de la especialización de los desarrolladores
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[(df_aux[col_freelance] == "A social science") | (df_aux[col_freelance] == "A business discipline") |
               (df_aux[col_freelance] == "Management information systems") | (df_aux[col_freelance] == "Psychology") |
               (df_aux[col_freelance] == "Information technology, networking, or system administration") |
               (df_aux[col_freelance] == "A humanities discipline"),col_freelance] = "social science"
    df_aux.loc[(df_aux[col_freelance] == "Computer science or software engineering") |
               (df_aux[col_freelance] == "Computer programming or Web development") |
               (df_aux[col_freelance] == "Computer engineering or electrical/electronics engineering") |
               (df_aux[col_freelance] == "Mathematics or statistics"),col_freelance] = "engineering,physics, mathematics or statistics"
    df_aux.loc[df_aux[col_freelance] == "A natural science",col_freelance] = "natural science"
    df_aux.loc[df_aux[col_freelance] == "A health science",col_freelance] = "health science"
    df_aux.loc[df_aux[col_freelance] == "Fine arts or performing arts",col_freelance] = "arts"
    df_aux.loc[(df_aux[col_freelance] == "I never declared a major") |(df_aux[col_freelance] == "Something else") |
              (df_aux[col_freelance] == "A non-computer-focused engineering discipline"),col_freelance] = "other"
    df_aux[col_freelance]=df_aux[col_freelance].fillna("sin info")
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out

def recode_Specialization_education(df_companies,col_Company_Name,col_companies):
    """
    Recodificación de la especialización requerida de las compañias
    """
    df_aux = df_companies[[col_Company_Name,col_companies]].copy()
    df_aux[col_companies]=df_aux[col_companies]
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(POL.SC|bussiness|Entertainment|human|psychology|journalism|marketing|markting|communic|finance|law|information|economics|Communication|business|politic|nutrition|sociology|dsign|design|history|media|management).*','social science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(aerospace|artificial intelligence|mining|technology|eng|Computer|math|physics).*','engineering, physics, mathematics or statistics')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(geology|Earth|Biology).*','natural science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(Biomedical|medicine|nurs).*','health science')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(arts|music|theatre).*','arts')
    df_aux[col_companies]=df_aux[col_companies].fillna("sin info")
    df_aux.loc[(df_aux[col_companies] != "arts") & (df_aux[col_companies] != "health science") &
               (df_aux[col_companies] != "natural science") & (df_aux[col_companies] != "sin info") &
               (df_aux[col_companies] != "social science") &
               (df_aux[col_companies] != "engineering, physics, mathematics or statistics"),col_companies] = "other"
    df_out=col_to_dummies(df_aux,col_companies)
    return df_out

In [9]:
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out2.head()

,Company_Name,Highest education_Bachelors,Highest education_Masters,Highest education_No Info,Highest education_PhD,under bachelor's degree
0,Company1,0.0,1.0,0.0,0.0,0.0
1,Company2,0.0,1.0,0.0,0.0,0.0
2,Company3,1.0,0.0,0.0,0.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0


In [10]:
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out2.head()

,Respondent,FormalEducation_Bachelors,FormalEducation_Masters,FormalEducation_No Info,FormalEducation_PhD,FormalEducation_under bachelor's degree
0,1,0.0,0.0,0.0,0.0,1.0
1,2,0.0,0.0,0.0,0.0,1.0
2,3,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0


In [11]:
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out3.head()

,Company_Name,Specialization of highest education_arts,"Specialization of highest education_engineering, physics, mathematics or statistics",Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science
0,Company1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Company2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Company3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Company4,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Company5,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out3.head()

,Respondent,MajorUndergrad_arts,"MajorUndergrad_engineering,physics, mathematics or statistics",MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science
0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Generación del dataset de compañías aplicando las funciones de educación

In [13]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_aux=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_aux,df_companies3,on='Company_Name',how='left')
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out=pd.merge(df_companies_out1,df_companies_out2,on='Company_Name',how='left')
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out=pd.merge(df_companies_out2,df_companies_out3,on='Company_Name',how='left')
df_companies_out.head()

,Company_Name,Highest education_Bachelors,Highest education_Masters,Highest education_No Info,Highest education_PhD,under bachelor's degree,Specialization of highest education_arts,"Specialization of highest education_engineering, physics, mathematics or statistics",Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science
0,Company1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Generando el dataset de freelances aplicando las funciones de educación

In [14]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out=pd.merge(df_freelance_out1,df_freelance_out2,on='Respondent',how='left')
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out=pd.merge(df_freelance_out2,df_freelance_out3,on='Respondent',how='left')
df_freelance_out.head()


,Respondent,FormalEducation_Bachelors,FormalEducation_Masters,FormalEducation_No Info,FormalEducation_PhD,FormalEducation_under bachelor's degree,MajorUndergrad_arts,"MajorUndergrad_engineering,physics, mathematics or statistics",MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science
0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


#### Funciones relacionadas con habilidades de los freelancers:

En el dataset de compañías la única recodificación es en las columnas que ya poseen porcentaje para cada uno de los skills, para establecerlas dentro de un rango adecuado.

En el dataset de freelancers la recodificación es mayor, ya que la primera recodificación es para establecer un valor númerico a las respuestas categoricas y la segunda es un ponderado entre algunas habilidades con el fin de crear un porcentaje que sea capaz de hacer match con el dataset de compañías.

In [15]:
def recode(companies,col_percent_skill):
    """
    Función para recodificar los intervalos de porcentaje en el dataset de compañias
    """
    df_comp=companies.copy()
    df_comp[col_percent_skill]=df_comp[col_percent_skill].replace('No Info',101)
    df_comp[col_percent_skill]=df_comp[col_percent_skill].astype(float)
    df_comp.loc[(df_comp[col_percent_skill] >=0 ) & (df_comp[col_percent_skill] < 20), col_percent_skill] = 0.2 
    df_comp.loc[(df_comp[col_percent_skill] >=20 ) & (df_comp[col_percent_skill] < 40), col_percent_skill] = 0.4
    df_comp.loc[(df_comp[col_percent_skill] >=40) & (df_comp[col_percent_skill] < 60), col_percent_skill] = 0.6
    df_comp.loc[(df_comp[col_percent_skill] >=60 ) & (df_comp[col_percent_skill] < 80), col_percent_skill] = 0.8
    df_comp.loc[(df_comp[col_percent_skill] >=80) & (df_comp[col_percent_skill] < 100), col_percent_skill] = 1
    df_comp.loc[(df_comp[col_percent_skill] == 101.0 ) , col_percent_skill] = 0
    
    return df_comp

def recode_other_skills(df_freelance,col_freelance):
    """
    Función que establece un valor porcentaje para el dataset de freelancers
    """
    df_aux = df_freelance.copy()
    df_aux.loc[df_aux[col_freelance] == "Strongly disagree",col_freelance] = 20
    df_aux.loc[df_aux[col_freelance] == "Disagree",col_freelance] = 40
    df_aux.loc[df_aux[col_freelance] == "Somewhat agree",col_freelance] = 60
    df_aux.loc[df_aux[col_freelance] == "Agree",col_freelance] = 80
    df_aux.loc[df_aux[col_freelance] == "Strongly agree",col_freelance] = 100
    
    return df_aux

In [16]:
#Funciones para unificar variables en el dataset de freelancers. 

def Skill_leadership(freelances_dataset):
    freelances_dataset['Skill_leadership'] = freelances_dataset[["RightWrongWay", "CompetePeers","DiversityImportant","ChallengeMyself"]].sum(axis=1, skipna = True)
    freelances_dataset['Percent_Skill_leadership']=(freelances_dataset['Skill_leadership']/4)
    return freelances_dataset

def Skill_Operations(freelances_dataset):
    freelances_dataset['Skill_Operations'] = freelances_dataset[["UnderstandComputers", "InvestTimeTools"]].sum(axis=1, skipna = True)
    freelances_dataset['Percent_Skill_Operations']=(freelances_dataset['Skill_Operations']/2)
    return freelances_dataset


In [17]:
var_comp_recode=["Company_Name","Percent_skill_Business Strategy","Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Leadership","Percent_skill_Engineering"]
var_freelances_recode=["Respondent","BuildingThings","LearningNewTech", "RightWrongWay","CompetePeers", "DiversityImportant","ChallengeMyself","UnderstandComputers","InvestTimeTools","ChangeWorld"]

def skills_comp(companies,var_comp_recode):
    companies_out=companies.copy()
    for i in var_comp_recode[1:]:
        companies_out = recode(companies_out[var_comp_recode],i)
    return companies_out
    
def skills_freel(freelances,var_freelances_recode):
    freelances_out=freelances[var_freelances_recode].copy()
    for x in var_freelances_recode[1:]:
        freelances_out = recode_other_skills(freelances_out[var_freelances_recode],x)
    freelances_out=Skill_leadership(freelances_out)    
    freelances_out=Skill_Operations(freelances_out)    
    return freelances_out


In [101]:
df_freelance_out4=skills_freel(freelances,var_freelances_recode)
df_freelance_out4=df_freelance_out4.drop(["UnderstandComputers", "InvestTimeTools","Skill_Operations"],axis=1)
df_freelance_out4=df_freelance_out4.drop(["RightWrongWay", "CompetePeers","DiversityImportant","ChallengeMyself","Skill_leadership"], axis=1)
df_freelance_out4

,Respondent,BuildingThings,LearningNewTech,ChangeWorld,Percent_Skill_leadership,Percent_Skill_Operations
0,1,100,80,80,65.0,70.0
1,2,NaN,NaN,NaN,0.0,0.0
2,3,100,100,80,65.0,50.0
3,4,100,100,80,75.0,50.0
4,5,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...
51387,51388,80,100,60,65.0,90.0
51388,51389,NaN,NaN,NaN,0.0,0.0
51389,51390,NaN,NaN,NaN,0.0,0.0
51390,51391,80,100,20,65.0,60.0


Al recodificar las habilidades del dataset de compañías es necesario reemplazar los valores entre 0 y y 1 a valores entre 0 y 100 para los porcentajes. 

In [171]:
df_companies_out4=skills_comp(companies,var_comp_recode)
df_companies_out4.head()
col_names=["Percent_skill_Business Strategy","Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Leadership","Percent_skill_Engineering"]

#Recodifica los valores entre 0 y 1 a valores entre 0 y 100 para aplicar las funciones del match 
for i in col_names:
        df_companies_out4[i]=df_companies_out4[i].replace(0.2,20)
        df_companies_out4[i]=df_companies_out4[i].replace(0.4,40)
        df_companies_out4[i]=df_companies_out4[i].replace(0.6,60)
        df_companies_out4[i]=df_companies_out4[i].replace(0.8,80)
        df_companies_out4[i]=df_companies_out4[i].replace(1,100)
        
df_freelance_out4=df_freelance_out4[["LearningNewTech","ChangeWorld","Percent_Skill_Operations","Percent_Skill_leadership","BuildingThings"]]
df_companies_out4.head()

,Company_Name,Percent_skill_Business Strategy,Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Leadership,Percent_skill_Engineering
0,Company1,20.0,20.0,20.0,20.0,20.0
1,Company2,40.0,20.0,20.0,20.0,20.0
2,Company3,20.0,20.0,20.0,20.0,60.0
3,Company4,20.0,20.0,20.0,20.0,20.0
4,Company5,20.0,20.0,20.0,20.0,20.0


Para el dataset de freelancers es necesario recodificar las columnas Percent_Skill_leadership y Percent_Skill_Operations ya que han quedado con valores fuera de rango como 65 o 75.

In [172]:
col_percent=["Percent_Skill_Operations","Percent_Skill_leadership"]

#Recodifica los valores entre 0 y 1 a valores entre 0 y 100 para aplicar las funciones del match 
for p in col_percent:
        df_freelance_out4[p]=df_freelance_out4[p].replace(5,20)
        df_freelance_out4[p]=df_freelance_out4[p].replace(10,20)
        df_freelance_out4[p]=df_freelance_out4[p].replace(15,20)
        df_freelance_out4[p]=df_freelance_out4[p].replace(25,40)
        df_freelance_out4[p]=df_freelance_out4[p].replace(30,40)
        df_freelance_out4[p]=df_freelance_out4[p].replace(35,40)
        df_freelance_out4[p]=df_freelance_out4[p].replace(45,60)
        df_freelance_out4[p]=df_freelance_out4[p].replace(50,60)
        df_freelance_out4[p]=df_freelance_out4[p].replace(55,60)
        df_freelance_out4[p]=df_freelance_out4[p].replace(65,80)
        df_freelance_out4[p]=df_freelance_out4[p].replace(70,80)
        df_freelance_out4[p]=df_freelance_out4[p].replace(75,80)
        df_freelance_out4[p]=df_freelance_out4[p].replace(85,100)
        df_freelance_out4[p]=df_freelance_out4[p].replace(90,100)
        df_freelance_out4[p]=df_freelance_out4[p].replace(95,100)
        
df_freelance_out4_f=df_freelance_out4

## Aplicación de todas las fuciones y unificación del dataset

In [173]:
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation").drop(["Respondent"], axis=1)
df_freelance_out=pd.concat([df_freelance_out1, df_freelance_out2.reindex(df_freelance_out1.index)], axis=1)
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad").drop(["Respondent"], axis=1)
df_freelance_out=pd.concat([df_freelance_out, df_freelance_out3.reindex(df_freelance_out.index)], axis=1)
df_freelance_out=pd.concat([df_freelance_out, df_freelance_out4_f.reindex(df_freelance_out.index)], axis=1)
freelances_dataset=pd.DataFrame(df_freelance_out)
df_freelance_out.head()


,Respondent,DeveloperType_Analytics,DeveloperType_Graphics Designer,DeveloperType_Mobile developer,DeveloperType_Web developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Database or system administrator,DeveloperType_Data scientist,DeveloperType_Machine learning specialist,...,MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science,LearningNewTech,ChangeWorld,Percent_Skill_Operations,Percent_Skill_leadership,BuildingThings
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,80,80,80.0,80.0,100
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN
2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100,80,60.0,80.0,100
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,100,80,60.0,80.0,100
4,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN


In [174]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_aux=pd.merge(df_companies1,df_companies2,on='Company_Name',how='left')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_aux,df_companies3,on='Company_Name',how='left')
df_companies_out2=recode_Highest_education(companies,"Company_Name","Highest education")
df_companies_out=pd.merge(df_companies_out1,df_companies_out2,on='Company_Name',how='left')
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out=pd.merge(df_companies_out,df_companies_out3,on='Company_Name',how='left')
df_companies_out=pd.merge(df_companies_out,df_companies_out4,on='Company_Name',how='left')
companies_dataset=df_companies_out
companies_dataset.head()

,Company_Name,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning,...,Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science,Percent_skill_Business Strategy,Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Leadership,Percent_skill_Engineering
0,Company1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,20.0,20.0,20.0,20.0,20.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,40.0,20.0,20.0,20.0,20.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,20.0,20.0,20.0,20.0,60.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,20.0,20.0,20.0,20.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,20.0,20.0,20.0,20.0


Imputando los NaN a zeros

In [175]:
#Distribución de los NaN

num_na_columns=pd.DataFrame({"Num NaN" : freelances_dataset.isna().sum(),
                            "Prop NaN": freelances_dataset.isna().sum() / len(freelances_dataset)}, 
                            columns=["Num NaN", "Prop NaN"])
num_na_columns.sort_values('Prop NaN', ascending=False)

,Num NaN,Prop NaN
ChangeWorld,20457,0.398058
BuildingThings,20158,0.392240
LearningNewTech,20088,0.390878
FormalEducation_under bachelor's degree,0,0.000000
Percent_Skill_leadership,0,0.000000
Percent_Skill_Operations,0,0.000000
MajorUndergrad_social science,0,0.000000
MajorUndergrad_sin info,0,0.000000
MajorUndergrad_other,0,0.000000
MajorUndergrad_natural science,0,0.000000


In [176]:
companies_dataset=companies_dataset.fillna(0)
companies_dataset.describe()

,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Database or system administrator,Data scientist,Machine Learning,Highest education_Bachelors,...,Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science,Percent_skill_Business Strategy,Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Leadership,Percent_skill_Engineering
count,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.0,472.000000,472.000000,472.000000,...,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000
mean,0.442797,0.093220,0.057203,0.027542,0.355932,0.023305,0.0,0.616525,0.273305,0.358051,...,0.008475,0.008475,0.288136,0.205508,0.275424,21.440678,18.644068,17.669492,17.881356,28.347458
std,0.497244,0.291049,0.232477,0.163831,0.479303,0.151031,0.0,0.486748,0.446129,0.479936,...,0.091764,0.091764,0.453375,0.404501,0.447202,11.857285,9.745401,7.290811,7.638510,22.246974
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,20.000000,20.000000,20.000000,20.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,20.000000,20.000000,20.000000,20.000000
75%,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,1.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,20.000000,20.000000,20.000000,20.000000,40.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,60.000000,100.000000,60.000000,60.000000,100.000000


In [177]:
num_na_companies=pd.DataFrame({"Num NaN" : companies_dataset.isna().sum(),
                            "Prop NaN": companies_dataset.isna().sum() / len(freelances_dataset)}, 
                            columns=["Num NaN", "Prop NaN"])
num_na_companies.sort_values('Prop NaN', ascending=False)

,Num NaN,Prop NaN
Company_Name,0,0.0
under bachelor's degree,0,0.0
Percent_skill_Leadership,0,0.0
Percent_skill_Operations,0,0.0
Percent_skill_Entrepreneurship,0,0.0
Percent_skill_Business Strategy,0,0.0
Specialization of highest education_social science,0,0.0
Specialization of highest education_sin info,0,0.0
Specialization of highest education_other,0,0.0
Specialization of highest education_natural science,0,0.0


In [178]:
freelances_dataset=freelances_dataset.fillna(0)
freelances_dataset.describe()

,Respondent,DeveloperType_Analytics,DeveloperType_Graphics Designer,DeveloperType_Mobile developer,DeveloperType_Web developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Database or system administrator,DeveloperType_Data scientist,DeveloperType_Machine learning specialist,...,MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science,LearningNewTech,ChangeWorld,Percent_Skill_Operations,Percent_Skill_leadership,BuildingThings
count,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,...,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000
mean,25696.500000,0.019945,0.006402,0.051234,0.510488,0.022844,0.070634,0.004923,0.008776,0.007686,...,0.002763,0.036290,0.066956,0.166388,0.105911,53.076743,42.816392,42.864259,46.311877,54.319349
std,14835.736854,0.139812,0.079755,0.220476,0.499895,0.149408,0.256214,0.069991,0.093268,0.087333,...,0.052493,0.187012,0.249948,0.372432,0.307727,44.042728,38.734419,36.443639,38.316866,44.898531
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12848.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25696.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,40.000000,60.000000,60.000000,80.000000
75%,38544.250000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,80.000000,80.000000,80.000000,100.000000
max,51392.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [179]:
num_na_freelances=pd.DataFrame({"Num NaN" : freelances_dataset.isna().sum(),
                            "Prop NaN": freelances_dataset.isna().sum() / len(freelances_dataset)}, 
                            columns=["Num NaN", "Prop NaN"])
num_na_freelances.sort_values('Prop NaN', ascending=False)

,Num NaN,Prop NaN
Respondent,0,0.0
FormalEducation_under bachelor's degree,0,0.0
Percent_Skill_leadership,0,0.0
Percent_Skill_Operations,0,0.0
ChangeWorld,0,0.0
LearningNewTech,0,0.0
MajorUndergrad_social science,0,0.0
MajorUndergrad_sin info,0,0.0
MajorUndergrad_other,0,0.0
MajorUndergrad_natural science,0,0.0


#### Generamos los dataset finales

In [180]:
freelances_dataset.to_excel("freelances_final.xlsx", index=False)
companies_dataset.to_excel("companies_final.xlsx", index=False)

In [181]:
freelances_dataset.to_csv("freelances_final.csv", index=False)
companies_dataset.to_csv("companies_final.csv", index=False)

In [182]:
f=["LearningNewTech","ChangeWorld","Percent_Skill_leadership","Percent_Skill_Operations","BuildingThings"]
freelancers_dummies=freelances_dataset.copy()
freelancers_dummies=col_to_dummies(freelancers_dummies,f[0])
freelancers_dummies=col_to_dummies(freelancers_dummies,f[1])
freelancers_dummies=col_to_dummies(freelancers_dummies,f[2])
freelancers_dummies=col_to_dummies(freelancers_dummies,f[3])
freelancers_dummies=col_to_dummies(freelancers_dummies,f[4])

In [183]:
c=["Percent_skill_Business Strategy","Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Leadership","Percent_skill_Engineering"]
companies_dummies=companies_dataset.copy()
companies_dummies=col_to_dummies(companies_dummies,c[0])
companies_dummies=col_to_dummies(companies_dummies,c[1])
companies_dummies=col_to_dummies(companies_dummies,c[2])
companies_dummies=col_to_dummies(companies_dummies,c[3])
companies_dummies=col_to_dummies(companies_dummies,c[4])


In [187]:
cols = list(companies_dummies.columns.values)
cols

['Company_Name',
 'Industry of company_Analytics',
 'Industry of company_Graphics Designer',
 'Industry of company_Mobile developer',
 'Industry of company_Web developer',
 'Industry of company_other',
 'Industry of company_software engineer',
 'Database or system administrator',
 'Data scientist',
 'Machine Learning',
 'Highest education_Bachelors',
 'Highest education_Masters',
 'Highest education_No Info',
 'Highest education_PhD',
 "under bachelor's degree",
 'Specialization of highest education_arts',
 'Specialization of highest education_engineering, physics, mathematics or statistics',
 'Specialization of highest education_health science',
 'Specialization of highest education_natural science',
 'Specialization of highest education_other',
 'Specialization of highest education_sin info',
 'Specialization of highest education_social science',
 'Percent_skill_Business Strategy_0.0',
 'Percent_skill_Business Strategy_20.0',
 'Percent_skill_Business Strategy_40.0',
 'Percent_skill_B

In [188]:
companies_dummies.to_excel("Companies_dummies.xlsx", index=False)
freelancers_dummies.to_excel("Freelancers_dummies.xlsx", index=False)